# **Using PuLP with Pandas and Binary Constraint to solve a scheduling problem**

In this example, we'll be solving a scheduling problem. We have 2 offshore production plants in 2 locations and an estimated demand for our products.

We want to produce a schedule of production from both plants that meets our demand with the lowest cost.

A factory can be in 2 states:
* Off - Producing zero units
* On - Producing between its minimum and maximum production capacities

Both factories have fixed costs, that are incurred as long as the factory is on, and variable costs, a cost per unit of production. These vary month by month. 

We also know that factory B is down for maintenance in month 5.

We will start by importing our data. The data is imported into a multi-index pandas Dataframe using "Month" and "Factory" as our indexed columns.

In [1]:
import pandas as pd
import pulp

In [2]:
factories = pd.DataFrame.from_csv("factory_variables.csv", index_col = ['Month', 'Factory'])
factories

C:\Users\chand\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


Max_Capacity  Min_Capacity  Variable_Costs  Fixed_Costs
Month Factory                                                         
1     A              100000         20000              10          500
      B               50000         20000               5          600
2     A              110000         20000              11          500
      B               55000         20000               4          600
3     A              120000         20000              12          500
      B               60000         20000               3          600
4     A              145000         20000               9          500
      B              100000         20000               5          600
5     A              160000         20000               8          500
      B                   0             0               0            0
6     A              140000         20000               8          500
      B               70000         20000               6          600
7     A              155000         20000               5          500
      B               60000         20000               4          600
8     A              200000         20000               7          500
      B              100000         20000               6          600
9     A              210000         20000               9          500
      B              100000         20000               8          600
10    A              197000         20000              10          500
      B              100000         20000              11          600
11    A               80000         20000               8          500
      B              120000         20000              10          600
12    A              150000         20000               8          500
      B              150000         20000              12          600

We will also import our demand data

In [3]:
demand = pd.DataFrame.from_csv('monthly_demand.csv', index_col = ['Month'])
demand

C:\Users\chand\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


,Demand
Month,
1,120000
2,100000
3,130000
4,130000
5,140000
6,130000
7,150000
8,170000
9,200000


As we have fixed costs and variable costs, we will need to model both production and the status of the factory i.e. whether it is on or off.

Production is modelled as an integer variable.

We have a value for production for each month for each factory, this is given by the tuples of our multi-index pandas DataFrame index.

In [4]:
production = pulp.LpVariable.dicts("production", ((month, factory) for month, factory in factories.index),
                                  lowBound = 0, cat = "Integer")

Factory status is modelled as a binary variable. It will have a value of 1 if the factory is on and a value of 0 when the factory is off.

Binary variables are the same as integer variables but constrained to be $\geq$ 0 and $\leq$ 1.

Again this has a value for each month for each factory, again given by the index of our DataFrame.

In [8]:
factory_status = pulp.LpVariable.dicts("factory_status", ((month, factory) for month, factory in factories.index),
                                       cat = "Binary")

We instantiate our model and use LpMaximize as the aim is to minimize costs.

In [ ]:
model = pulp.LpProblem("Cost minimizing scheduling problem", 
                      pulp.LpMinimize)

In our objective function we include our 2 costs:
* Our variable costs is the product of the variable costs per unit and production. 
* Our fixed costs is the factory status -1 (On) or 0 (Off) - mulitplied by the fixed cost of production

In [9]:
model += pulp.lpSum([production[month, factory] * factories.loc[(month, factory), "Variable_Costs"]
                    for month, factory in factories.index]
                   +
                   [factory_status[month, factory] * factories.loc[(month, factory), "Fixed_Costs"]
                   for month, factory in factories.index])

We build up our constraints

In [11]:
#Production in any month must be equal to demand
months = demand.index
for month in months:
    model += production[(month, "A")] + production[(month, "B")] == demand.loc[month, "Demand"]

An issue we run out here is that in Linear programming we can't use conditional constraints.

For example, we can't add to our model that if the factory is off factory status must be 0, and if it is on factory status must be 1. Before we've solved our model though, we don't know if the factory will be on or off in a given month.

In this case construct constraints that have minimum and maximum capacities that are constant variables, which we multiply by the factory status. 

Now, either factory status is 0 and:
* min_production $\geq$ 0
* max_production $\leq$ 0

Or factory status is 1 and:
* min_production $\geq$ min_capacity
* max_production $\leq$ max_capacity

*(In some cases we can use linear constraints to model conditional statements)*

In [12]:
#Production in any month must be between minimum and maximum capacity, or zero

for month, factory in factories.index:
    min_production = factories.loc[(month, factory), "Min_Capacity"]
    max_production = factories.loc[(month, factory), "Max_Capacity"]
    model += production[(month, factory)] >= min_production * factory_status[month, factory]
    model += production[(month, factory)] <= max_production * factory_status[month, factory]

In [13]:
#Factory B is off in May
model += factory_status[5, "B"] == 0
model += production[5, "B"] == 0

We then solve the model

In [15]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

Let's took a look at the optimal production schedule output for each month from each factory. For ease of viewing we'll output the data to a pandas DataFrame.

In [16]:
output = []
for month, factory in production:
    var_output = {
        "Month": month, 
        "Factory": factory,
        "Production": production[(month, factory)].varValue,
        "Factory Status": factory_status[(month, factory)].varValue
    }
    output.append(var_output)
output_df = pd.DataFrame.from_records(output).sort_values(['Month', 'Factory'])
output_df.set_index(['Month', 'Factory'], inplace = True)
output_df

Factory Status  Production
Month Factory                            
1     A                   1.0     70000.0
      B                   1.0     50000.0
2     A                   1.0     45000.0
      B                   1.0     55000.0
3     A                   1.0     70000.0
      B                   1.0     60000.0
4     A                   1.0     30000.0
      B                   1.0    100000.0
5     A                   1.0    140000.0
      B                   0.0         0.0
6     A                   1.0     60000.0
      B                   1.0     70000.0
7     A                   1.0     90000.0
      B                   1.0     60000.0
8     A                   1.0     70000.0
      B                   1.0    100000.0
9     A                   1.0    100000.0
      B                   1.0    100000.0
10    A                   1.0    190000.0
      B                   0.0         0.0
11    A                   1.0     80000.0
      B                   1.0     60000.0
12    A                   1.0    100000.0
      B                   0.0         0.0

Notice above that the factory status is 0 when not producing and 1 when it is producing

In [17]:
#Print our objective function value (Total Costs)
print(pulp.value(model.objective))

12906400.0


# **Mocking conditional statements using Binary Constraints**

We will explore not only conditional statements using binary constraints, but combining them with logical operators, 'and' and 'or'.

First, we'll work through some theory, then a real world example as an extension of part 5's example at the end.

**Conditional statement**

To start simply, if we have the binary constraint $x_1$ and we want:

```
if x1 == 1:
   y1 == 1
elif x1 == 0:
   y1 == 0 
```
    
We can achieve this easily using the following constraint:
y1 == x1


However, if we wanted the opposite:
```
if x1 == 1:
   y1 == 0
elif x1 == 0:
   y1 == 1 
```

Given that they most both be 1 or 0, we just need the following constraint:

x1 + y1 == 1

**Logical 'AND' operator**

Now for something a little more complex, we can coerce a particular binary constraint to be 1 based on the states of 2 other binary constraints.

If we have binary constraints $x_1$ and $x_2$ and we want to achieve the following:

```
if x1 == 1 and x2 == 0:
    y1 == 1
else:
    y1 == 0
```

So that $y_1$ is only 1 in the case that $x_1$ is 1 and $x_2$ is 0. We can use the following 3 constraints to achieve this:

y1 $\geq$ x1 - x2,

y1 $\leq$ x1,

y1 $\leq$ (1 - x2)

We'll take a moment to deconstruct this. In our preferred case that x1 = 1 and x2 = 0, the three statments resolve to:

* y1 ≥ 1

* y1 ≤ 1

* y1 ≤ 1

The only value of  𝑦1  that fulfils each of these is 1.

In any other case, however, y1 will be zero. Let's take another example, say x1 = 0 and x2 = 1. This resolves to:

* y1 ≥ -1

* y1 ≤ 0

* y1 ≤ 0

Given that y1 is a binary variable and must be 0 or 1, the only value of y1 that can fulfil each of these is 0.

You can construct 3 constraints so that y1 is equal to 1, only in the case you're interested in out of the 4 following options:

* x1 = 1 and x2 = 1

* x1 = 1 and x2 = 0

* x1 = 0 and x2 = 1

* x1 = 0 and x2 = 0

I have created a function for exactly this purpose to cover all cases:

In [20]:
def make_io_and_constraint(y1, x1, x2, target_x1, target_x2):
    """
    Returns a list of constraints for a linear programming model that 
    will constrain y1 to 1 when
    x1 = target_x1 and x2 = target_x2;
    where target_x1 and target_x2 are 1 and 0
    """
    binary = [0, 1]
    assert target_x1 in binary
    assert target_x2 in binary
    
    if IOx1 == 1 and IOx2 == 1:
        return [
            y1 >= x1 + x2 - 1,
            y1 <= x1,
            y1 <= x2
        ]
    elif IOx1 == 1 and IOx2 == 0:
        return [
            y1 >= x1 - x2,
            y1 <= x1,
            y1 <= (1 - x2)
        ]
    elif IOx1 == 0 and IOx2 == 1:
        return [
            y1 >= x2 - x1,
            y1 <= (1 - x1),
            y1 <= x2
        ]
    else:
        return [
            y1 >= - (x1 + x2 -1),
            y1 <= (1 - x1),
            y1 <= (1 - x2)
        ]

**Logical 'OR' operator**

This is all well and good for the 'and' logical operator. What about the 'or' logical operator.

If we would like the following:

```
if x1 == 1 or x2 == 1:
    y1 == 1
else:
    y1 == 0
```

We can use the following linear constraints:

    y1 <= x1 + x2
    
    y1 * 2 >= x1 + x2

So that:

* if x1 is 1 and x2 is 1:
    * y1 ≤ 2
    * 2y1 ≥ 2
    * y1 must equal 1
* if x1 is 1 and x2 is 0:
    * y1 ≤ 1
    * 2y1 ≥ 1
    * y1 must equal 1
* if x1 is 0 and x2 is 1:
    * y1 ≤ 1
    * 2y1 ≥ 1
    * y1 must equal 1
* if x1 is 0 and x2 is 0:
    * y1 ≤ 0
    * 2y1 ≥ 0
    * y1 must equal 0
    
Again, we'll consider the alternative option:

```
if x1 == 0 or x2 == 0:
    y1 == 1
else:
    y1 == 0
```

We can use the following linear constraints:

    y1 * 2 <= 2 - (x1 + x2)
    
    y1 >= 1 - (x1 + x2)

**Example - Scheduling Example Extended**

Both factories had 2 costs:

* Fixed Costs - Costs incurred while the factory is running
* Variable Costs - Cost per unit of production
We're going to introduce a third cost - Start up cost.

This will be a cost incurred by turning on the machines at one of the factories.

In this example, our start-up costs will be:

* Factory A - \$20,000
* Factory B - \$400,000


We'll begin by defining our decision variables, we have an additional binary variable for switching on thr factory.

In [21]:
#Production
production = pulp.LpVariable.dicts("production",
                                  ((month, factory) for month, factory in factories.index),
                                  lowBound = 0,
                                  cat = "Integer")

# Factory Status, On or Off
factory_status = pulp.LpVariable.dicts("factory_status",
                                      ((month, factory) for month, factory in factories.index),
                                      cat = "Binary")

# Factory switch on or off
switch_on = pulp.LpVariable.dicts("switch_on",
                                 ((month, factory) for month, factory in factories.index),
                                 cat = "Binary")

We instantiate our model and define our objective function, including start up costs

In [22]:
# Instantiate the model
model = pulp.LpProblem("Cost minimizing scheduling problem", pulp.LpMinimize)

# Select index on factory A or B
factory_A_index = [tpl for tpl in factories.index if tpl[1] == 'A']
factory_B_index = [tpl for tpl in factories.index if tpl[1] == 'B']

# Define objective function
model += pulp.lpSum(
    [production[m, f]*factories.loc[(m, f), 'Variable_Costs'] for m, f in factories.index]
    + [factory_status[m, f] * factories.loc[(m, f), "Fixed_Costs"] for m, f in factories.index]
    + [switch_on[m, f]*20000 for m, f in factory_A_index]
    + [switch_on[m, f]*400000 for m, f in factory_B_index]
)

In [24]:
# Production in any month must be equal to demand
months = demand.index
for month in months:
    model += production[(month, "A")] + production[(month, "B")] == demand.loc[month, "Demand"]
    
# Production in any month must be between minimum and maximum capacity, or zero
for month, factory in factories.index:
    min_production = factories.loc[(month, factory), "Min_Capacity"]
    max_production = factories.loc[(month, factory), "Max_Capacity"]
    model += production[(month, factory)] >= min_production * factory_status[month, factory]
    model += production[(month, factory)] <= max_production * factory_status[month, factory]
    
# Factory B is off in May
model += factory_status[5, "B"] == 0
model += production[5, "B"] == 0

Now we want to add in our constraints for switching on.

A factory switches on if:
* It is off in the previous month (m-1)
* AND it is on in the current month (m).

As we don't know if the factory is on before month 0, we'll assume that the factory has switched on if it's on in month 1.

In [25]:
for month, factory in factories.index:
    # In month 1, if the factory ison, we assume it turned on
    if month == 1:
        model += switch_on[month, factory] == factory_status[month, factory]
    
    # In other months, if the factory is on in the current month AND off in the previous month, switch on = 1
    else:
        model += switch_on[month, factory] >= factory_status[month, factory] - factory_status[month-1, factory]
        model += switch_on[month, factory] <= 1 - factory_status[month-1, factory]
        model += switch_on[month, factory] <= factory_status[month, factory]

In [26]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [27]:
output = []
for month, factory in production:
    var_output = {
        'Month': month,
        'Factory': factory,
        'Production': production[(month, factory)].varValue,
        'Factory Status': factory_status[(month, factory)].varValue,
        'Switch On': switch_on[(month, factory)].varValue
    }
    output.append(var_output)
output_df = pd.DataFrame.from_records(output).sort_values(['Month', 'Factory'])
output_df.set_index(['Month', 'Factory'], inplace=True)
output_df

Factory Status  Production  Switch On
Month Factory                                       
1     A                   1.0     70000.0        1.0
      B                   1.0     50000.0        1.0
2     A                   1.0     45000.0        0.0
      B                   1.0     55000.0        0.0
3     A                   1.0     70000.0        0.0
      B                   1.0     60000.0        0.0
4     A                   1.0     30000.0        0.0
      B                   1.0    100000.0        0.0
5     A                   1.0    140000.0        0.0
      B                   0.0         0.0        0.0
6     A                   1.0     60000.0        0.0
      B                   1.0     70000.0        1.0
7     A                   1.0     90000.0        0.0
      B                   1.0     60000.0        0.0
8     A                   1.0     70000.0        0.0
      B                   1.0    100000.0        0.0
9     A                   1.0    100000.0        0.0
      B                   1.0    100000.0        0.0
10    A                   1.0    170000.0        0.0
      B                   1.0     20000.0        0.0
11    A                   1.0     80000.0        0.0
      B                   1.0     60000.0        0.0
12    A                   1.0    100000.0        0.0
      B                   0.0         0.0        0.0

Interestingly, we see that it now makes economic sense to keep factory B on after it turns off in month 5 up until month 12.

Previously, we had the case that it was not economic to run factory B in month 10, but as there is now a significant cost to switching off and back on, the factory runs through month 10 at its lowest capacity (20,000 units).